In [9]:
# import Pkg

# package_list = ["DataFrames", "CSV",
#  "JuMP", "Gurobi",
#  "LinearAlgebra", "Random", "Printf", "StatsBase", "CategoricalArrays",
#  "Plots", "StatsPlots",
#  "Distributions", "JLD"]

# for pkg_name in package_list
#     Pkg.update(pkg_name)
#     Pkg.add(pkg_name)
# end

In [10]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [11]:
liverpool_data = CSV.read("Liverpool_FIFA_22.csv", DataFrame)
n_players = nrow(liverpool_data)

33

In [12]:
all_teams = CSV.read("Team_Ratings.csv", DataFrame)
n_games = 2*(nrow(all_teams)-1)
n_positions = 27

27

In [13]:
GK = "gk"
defenders = ["lwb", "rwb", "lb", "lcb", "cb", "rcb", "rb"]
midfielders = ["lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "ldm", "cdm", "rdm"]
forwards = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw"]
center_backs = ["lcb", "cb", "rcb"]
right_backs = ["rwb", "rb"]
left_backs = ["lwb", "lb"]
cdms = ["ldm", "cdm", "rdm"]
cms = ["lcm", "cm", "rcm"]
cams = ["lam", "cam", "ram"]
left_wingers = ["lw", "lm"]
right_wingers = ["rw", "rm"]
strikers = ["ls", "st", "rs", "lf", "cf", "rf"]

6-element Vector{String}:
 "ls"
 "st"
 "rs"
 "lf"
 "cf"
 "rf"

In [20]:
player_pos = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw", "lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "lwb", "ldm", "cdm", "rdm", "rwb", "lb", "lcb", "cb", "rcb", "rb", "gk"]
pos_to_ind = Dict()
index = 1
for pos in player_pos
    pos_to_ind[pos] = index
    index += 1
end
pos_to_ind["st"]

2

In [21]:
model = Model(with_optimizer(Gurobi.Optimizer))

### Decision Variables ###

### x[i][j][k]. player i, game j, position k ###
@variable(model, x[i=1:n_players, j=1:n_games, k=1:n_positions], Bin)

### z variable, for whether we go for win or not ###
@variable(model, z[j = 1:n_games], Bin)


### Match Winning Constraint ###
### R is player ratings, R[i][k] is player i for position k ###
### C is the opposing team rating, C[j] is the rating of the opposing team for game j ###
for j in 1:n_games
    @constraint(model, sum(sum(R[i][k]*x[i][j][k] for i in 1:n_players) for k in 1:n_positions)/11 >= C[j]*z[j])
end

### Player Constraints ###

### Each player can only play at most once in a game ###
for j in 1:n_games
    for i in 1:n_players
        @constraint(model, sum(x[i, j, k] for k in 1:n_positions) <= 1)
    end
end

### Can only have 11 players in game ###
for j in 1:n_games
    @constraint(model, sum(sum(x[i, j, k] for k in 1:n_positions) for i in 1:n_players) == 11)

    ### 1 GK per game ###
    @constraint(model, sum(x[i, j, pos_to_ind[GK]] for i in 1:n_players) == 1)

    ### 4 Defenders per game ###
    for k in defenders
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 4)
    end

    ### 3 - 5 Midfielders per game ### 
    for k in midfielders
        @constraint(model, 3 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 5)
    end

    ### 1 - 3 Forwards per game ###
    for k in forwards
        @constraint(model, 1 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 3)
    end

    ### 2 CBs per game ###
    for k in center_backs
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 2)
    end

    ### 1 RB per game ###
    for k in right_backs
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 1)
    end

    ### 1 LB per game ###
    for k in left_backs
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 1)
    end

    ### 0 - 2 CDMs per game ###
    for k in cdms
        @constraint(model, 0 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 2)
    end

    ### 0 - 3 CMs per game ###
    for k in cms
        @constraint(model, 0 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 3)
    end

    ### 0 - 3 CAMS per game ###
    for k in cams
        @constraint(model, 0 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 3)
    end

    ### 0 - 1 LW, LM per game ###
    for k in left_wingers
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 1)
    end

    ### 0 - 1 RW, RM per game ###
    for k in right_wingers
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 1)
    end

    ### 1 ST per game ###
    for k in strikers
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 1)
    end 
end

### Objective Function ###
@Objective(model, Max, sum(z[j] for j in 1:n_games))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-28
